In [1]:
from flask import Flask, render_template, request, jsonify
import os
import pygame
from pygame.locals import *

# Initialize Flask app
app = Flask(__name__)
UPLOAD_FOLDER = 'uploads'
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

# Global variables for UI state
screen_width, screen_height = 800, 600
grid_size = 50
pieces = []  # Placeholder for pieces
selected_piece = None

# Initialize Pygame
pygame.init()
screen = pygame.display.set_mode((screen_width, screen_height))
pygame.display.set_caption('Pentomino UI')
clock = pygame.time.Clock()

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
GRAY = (200, 200, 200)
RED = (255, 0, 0)

# Functions to draw grid and pieces
def draw_grid():
    for x in range(0, screen_width, grid_size):
        pygame.draw.line(screen, GRAY, (x, 0), (x, screen_height))
    for y in range(0, screen_height, grid_size):
        pygame.draw.line(screen, GRAY, (0, y), (screen_width, y))

def draw_pieces():
    for piece in pieces:
        rect = pygame.Rect(piece['x'], piece['y'], grid_size, grid_size)
        pygame.draw.rect(screen, piece['color'], rect)
        if piece == selected_piece:
            pygame.draw.rect(screen, RED, rect, 2)

# Main game loop for Pygame
def game_loop():
    global selected_piece
    running = True

    while running:
        screen.fill(WHITE)
        draw_grid()
        draw_pieces()

        for event in pygame.event.get():
            if event.type == QUIT:
                running = False

            elif event.type == MOUSEBUTTONDOWN:
                mouse_x, mouse_y = event.pos
                for piece in pieces:
                    rect = pygame.Rect(piece['x'], piece['y'], grid_size, grid_size)
                    if rect.collidepoint(mouse_x, mouse_y):
                        selected_piece = piece
                        break

            elif event.type == MOUSEBUTTONUP:
                if selected_piece:
                    selected_piece['x'] = round(selected_piece['x'] / grid_size) * grid_size
                    selected_piece['y'] = round(selected_piece['y'] / grid_size) * grid_size
                    selected_piece = None

            elif event.type == MOUSEMOTION:
                if selected_piece:
                    selected_piece['x'] += event.rel[0]
                    selected_piece['y'] += event.rel[1]

            elif event.type == KEYDOWN:
                if selected_piece:
                    if event.key == K_r:  # Rotate
                        selected_piece['rotation'] = (selected_piece['rotation'] + 90) % 360
                    elif event.key == K_f:  # Flip
                        selected_piece['flipped'] = not selected_piece['flipped']

        pygame.display.flip()
        clock.tick(30)

    pygame.quit()

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/upload', methods=['POST'])
def upload_image():
    if 'file' not in request.files:
        return jsonify({'error': 'No file uploaded'}), 400

    file = request.files['file']
    if file.filename == '':
        return jsonify({'error': 'No selected file'}), 400

    if file:
        filepath = os.path.join(app.config['UPLOAD_FOLDER'], file.filename)
        file.save(filepath)
        return jsonify({'filepath': filepath})

    return jsonify({'error': 'File upload failed'}), 500

if __name__ == "__main__":
    # Start Pygame loop
    game_loop()


pygame 2.6.1 (SDL 2.28.4, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


KeyboardInterrupt: 